# Playground for Riot Watcher

This is a testing notebook where I explore and test out features of `riotwatcher`. 

In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import zilean
from zilean.snapshots import SnapShots

import json

api_key = zilean.read_api_key()
watcher = LolWatcher(api_key=api_key)
my_region = 'na1'
me = watcher.summoner.by_name(my_region, 'yENatividad')

## Match Timeline Summary

In [2]:
my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'])
latest_matchid = my_matches[2]
latest_match = watcher.match.by_id(my_region, latest_matchid)
latest_timeline = watcher.match.timeline_by_match(my_region, latest_matchid)

In [3]:
snap = SnapShots(latest_timeline, [8, 10], latest_matchid)
pd.DataFrame([snap.summary()])

,level_0_frame8,timeEnemySpentControlled_0_frame8,totalGold_0_frame8,xp_0_frame8,totalDamageDone_0_frame8,totalDamageDoneToChampions_0_frame8,totalDamageTaken_0_frame8,creepScore_0_frame8,level_1_frame8,timeEnemySpentControlled_1_frame8,...,level_4_frame10,timeEnemySpentControlled_4_frame10,totalGold_4_frame10,xp_4_frame10,totalDamageDone_4_frame10,totalDamageDoneToChampions_4_frame10,totalDamageTaken_4_frame10,creepScore_4_frame10,matchId,win
0,-1,2121,-854,-855,-5051,-1807,1246,-27,0,120691,...,0,-20745,-457,237,2448,552,318,7,NA1_4307172207,False


In [4]:
pd.DataFrame(snap.frame_independent_summary())

,level_0,timeEnemySpentControlled_0,totalGold_0,xp_0,totalDamageDone_0,totalDamageDoneToChampions_0,totalDamageTaken_0,creepScore_0,level_1,timeEnemySpentControlled_1,...,timeEnemySpentControlled_4,totalGold_4,xp_4,totalDamageDone_4,totalDamageDoneToChampions_4,totalDamageTaken_4,creepScore_4,matchId,win,frame
0,-1,2121,-854,-855,-5051,-1807,1246,-27,0,120691,...,-13690,-580,-257,1022,331,555,0,NA1_4307172207,False,8
1,-1,-26475,-987,-1003,-4996,-2461,1532,-36,-1,95889,...,-20745,-457,237,2448,552,318,7,NA1_4307172207,False,10
